In [1]:


"""
Created on Mon May 24 13:26:13 2021

## author: Kevin Sanchez
## contact: kevin.j.sanchez@nasa.gov
## project: Satillite_Contrail_Unet
## status: Complete, future changes possible.

Description:
    Text
    
    Arguments:
        arg1 -- arg1 description (format)
        arg2 -- (format)
        ...
    Returns: 
        data1, data2 --  (format, format)
            
        
"""



import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib import pyplot as plt
%matplotlib tk
import numpy as np
import struct
import os
import glob
import sys


dataDirPath = os.getcwd()
projPath = os.path.normpath(dataDirPath + os.sep + os.pardir) #get parent directory
print(projPath)
dataDirPath = os.path.join(projPath, 'data/')
print(dataDirPath)

/home/kjsanche/Desktop/Projects/Sat_Contrail_Unet
/home/kjsanche/Desktop/Projects/Sat_Contrail_Unet/data/


In [2]:
# obtain satellite image and mask paths
imageNum = 3 #which image to work on.

auxList = glob.glob(dataDirPath + "/**/*01__1km.AUX", recursive = True)
granulePath = os.path.normpath(auxList[imageNum] + os.sep + os.pardir) #get parent directory

satImage11 = glob.glob(granulePath + "/31__1km.raw", recursive = True)
satImage12 = glob.glob(granulePath + "/32__1km.raw", recursive = True)

maskFile = glob.glob(granulePath + "/*.contrail-mask", recursive = True)
maskFile = [ x for x in maskFile if "_sw" not in x ] #remove _sw files

saveImgPath = maskFile[0].split('.contrail-mask')[0]

cntWrites = 0

print('Granule Path:' + saveImgPath)
if os.path.isfile(saveImgPath + '_finalMask.png'):
    queryOverwrite = input('Do you want to overwrite (o) or load (l) existing updated mask or neither (n)? (o/l/n)')
    if queryOverwrite != 'o' and queryOverwrite != 'l':
        sys.exit("User choose to not overwrite existing updated mask")
else:
    queryOverwrite = 'N/A'


Granule Path:/home/kjsanche/Desktop/Projects/Sat_Contrail_Unet/data/093/A2019093.1445/MYD021KM-A2019093.1445


In [3]:
def onClick(event):
    global clickpt, mask, imgDim
    ix, iy = event.xdata, event.ydata
    
    #make polygon and set values within to 0
    if ix == None:
        #use clicks to make polygon   
        x, y = np.meshgrid(np.arange(imgDim[0]), np.arange(imgDim[1])) # make a canvas with coordinates
        x, y = x.flatten(), y.flatten()
        points = np.vstack((x,y)).T     
        p = Path(clickpt)
        grid = p.contains_points(points)
        
        #make mask where values in polygon are set to 0
        polygonMask = grid.reshape(imgDim[1],imgDim[0]) # now we have a mask with points inside a polygon
        mask[polygonMask] = 0
        
        #update figure
        maskImg.set_data(mask) # change variable in maskimg
        fig.canvas.draw()
        fig.canvas.flush_events()
        clickpt = [] #reset click values
        return
    
    # append click points
    clickpt.append((ix, iy)) 

    print(clickpt, '\n')

    return

def shiftFigZoom(imgDimSize, zoomUpperVal,shiftAmt):
    if imgDimSize > zoomUpperVal+shiftAmt:
        zoomUpperVal += shiftAmt
    elif imgDimSize < zoomUpperVal+shiftAmt:
        zoomUpperVal = imgDimSize
    return zoomUpperVal


def toggleSaveImages(event):
    global mask, maskFile, cntWrites, imgDim, lastImgDim, saveImgPath
    
    #toggle the visible state of the sateillite and mask images
    if event.key == '9':
        b1 = satImg.get_visible()
        b2 = maskImg.get_visible()
        satImg.set_visible(not b1)
        maskImg.set_visible(not b2)
        plt.draw()
        
    #write new mask to file    
    if event.key == 'W':
        cntWrites += 1
        with open(maskFile[0] + 'Update' + str(cntWrites), 'wb') as f:
            np.save(f, mask)
        with open(maskFile[0] + 'Update', 'wb') as f:
            np.save(f, mask)
            
    #load new mask from file        
    if event.key == 'O':
        if cntWrites > 0:
            with open(maskFile[0] + 'Update' + str(cntWrites), 'rb') as f:
                mask = np.load(f)
            cntWrites -= 1
            maskImg.set_data(mask)
            fig.canvas.draw()
            fig.canvas.flush_events()
        
    #show zoomed out view        
    if event.key == '0':
        plt.axis([0, imgDim[0], imgDim[1], 0])
        imgDim[2:3] = [0,0]
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    #switch to previous image/zoomed view
    if event.key == '-':
        imgDim = lastImgDim[:]
        plt.axis([max(imgDim[2]-2000,0), imgDim[2], imgDim[3], max(imgDim[3]-1000,0)])
        fig.canvas.draw()
        fig.canvas.flush_events()
        
    #close figure, remove old updates
    if event.key == 'C':
        maskImg.set_visible(True)
        plt.draw()
        fig.savefig(saveImgPath + '_finalMask.png')
        plt.close()
        while cntWrites > 0:
            os.remove(maskFile[0] + 'Update' + str(cntWrites))
            cntWrites -= 1
        
    #zoom into image and shift zoom each time
    if event.key == '=':
        lastImgDim = imgDim[:]

        if imgDim[2] == 0 and imgDim[3] == 0: #first zoom
            imgDim[2] = 2000
            imgDim[3] = 1000
        elif imgDim[0] != imgDim[2]:
            imgDim[2] = shiftFigZoom(imgDim[0], imgDim[2], 1600) #shift in x direction 2000
        elif imgDim[0] == imgDim[2] and imgDim[1] != imgDim[3]:
            imgDim[2] = 2000
            imgDim[3] = shiftFigZoom(imgDim[1], imgDim[3], 800) #shift in y direction 1000
  
        #apply zoom in
        plt.axis([max(imgDim[2]-2000,0), imgDim[2], imgDim[3], max(imgDim[3]-1000,0)])
        fig.canvas.draw()
        fig.canvas.flush_events()
    return



def readContrailMask(path, imgDim):
    with open(path, mode='rb') as file: # b is important -> binary
        fileContent = file.read()
        data = np.uint16(np.reshape(struct.unpack("B"*imgDim[0]*imgDim[1], fileContent),(imgDim[1],imgDim[0])))
    return data
    
    
def readSatelliteImage(path, imgDim):
    with open(path, mode='rb') as file: # b is important -> binary
        fileContent = file.read()
        data = np.uint32(np.reshape(struct.unpack("H"*imgDim[0]*imgDim[1] , fileContent),(imgDim[1],imgDim[0])))
    return data


def getImageDimentions(path):
    f = open(path, "r")
    #print(f.read())
    for line in f:
        if "RawDefinition" in line:
            linestr = line.split()
            dim = [int(linestr[1]), int(linestr[2])]
            break
    f.close()
    return dim

In [4]:
imgDim = getImageDimentions(auxList[imageNum])

# Read a mask file (or load updated mask file for futher changes)
if queryOverwrite == 'l':
    with open(maskFile[0] + 'Update', 'rb') as f:
        mask = np.load(f)
else:
    mask = readContrailMask(maskFile[0],imgDim)


# Read the corresponding satellite image files and take difference between 11 and 12 micron images 
satImg11 = readSatelliteImage(satImage11[0],imgDim)
satImg12 = readSatelliteImage(satImage12[0],imgDim)
satImg = satImg11.astype(float)/100-satImg12.astype(float)/100

#for zoom in x and y extent to be updated later
imgDim.extend([0, 0]) 
lastImgDim = imgDim[:]



In [5]:

# Initialize clickpt to empty list, plot figures and set up click and toggle events
clickpt= []

fig = plt.figure()
ax1 = plt.axes([.04,.04,.95,.95])


cid = fig.canvas.mpl_connect('button_press_event', onClick)
cid2 = fig.canvas.mpl_connect('key_press_event', toggleSaveImages)

satImg = plt.imshow(satImg, cmap = 'gray')
manager = plt.get_current_fig_manager()
manager.resize(*manager.window.maxsize())
plt.show()
if not os.path.isfile(saveImgPath + '_visImg.png'):
    fig.savefig(saveImgPath + '_visImg.png')
maskImg = plt.imshow(mask, cmap = 'gray')
if not os.path.isfile(saveImgPath + '_origMask.png'):
    fig.savefig(saveImgPath + '_origMask.png')

#plt.colorbar(satImg)
#plt.colorbar(maskImg)
satImg.set_visible(False)



[(1126.999316473001, 629.4121667805878)] 

[(1126.999316473001, 629.4121667805878), (1001.2303485987699, 674.5249487354749)] 

[(1126.999316473001, 629.4121667805878), (1001.2303485987699, 674.5249487354749), (812.5768967874233, 681.3602187286397)] 

[(1126.999316473001, 629.4121667805878), (1001.2303485987699, 674.5249487354749), (812.5768967874233, 681.3602187286397), (853.5885167464116, 593.868762816131)] 

[(1126.999316473001, 629.4121667805878), (1001.2303485987699, 674.5249487354749), (812.5768967874233, 681.3602187286397), (853.5885167464116, 593.868762816131), (1073.684210526316, 611.6404647983594)] 

[(716.8831168831169, 632.1462747778536)] 

[(716.8831168831169, 632.1462747778536), (707.3137388926864, 633.5133287764866)] 

[(716.8831168831169, 632.1462747778536), (707.3137388926864, 633.5133287764866), (707.3137388926864, 649.917976760082)] 

[(716.8831168831169, 632.1462747778536), (707.3137388926864, 633.5133287764866), (707.3137388926864, 649.917976760082), (809.8427887901